In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# データの列名を定義
column_names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
                'acceleration', 'model year', 'origin', 'car name']

# データの読み込み
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
df = pd.read_csv(url, names=column_names, sep=r'\s+', na_values='?')

# 欠損値の処理
df = df.dropna().reset_index(drop=True)

df

In [ ]:
# 特徴量のリストから'origin'を除外
features = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
            'acceleration', 'model year']
X = df[features].values

# 'origin'のデータを取得（プロット用）
origins = df['origin'].values

# 平均と標準偏差を計算
mean = np.mean(X, axis=0)
std = np.std(X, axis=0)

# 標準化
X_std = (X - mean) / std

# 共分散行列の計算
cov_matrix = np.cov(X_std, rowvar=False)

# 固有値と固有ベクトルの計算
eigen_values, eigen_vectors = np.linalg.eig(cov_matrix)

# 固有値の降順でソート
sorted_index = np.argsort(eigen_values)[::-1]
eigen_values = eigen_values[sorted_index]
eigen_vectors = eigen_vectors[:, sorted_index]

# 固有値を表示
print('Eigen values: ', eigen_values)

# 固有ベクトルをdfへ変換
ev = pd.DataFrame(eigen_vectors, columns=[f'PC{i+1}' for i in range(7)], index=features)

# 固有ベクトルを表示
print(ev)

In [ ]:
# 上位2つの主成分を選択
k = 2
eigenvector_subset = eigen_vectors[:, 0:k]

# データを主成分空間に変換
X_reduced = np.dot(X_std, eigenvector_subset)

# マーカーの定義（原産国ごとに異なる形状）
markers = {1: 'o', 2: 's', 3: '^'}
place = {1: 'US', 2: 'Europe', 3: 'Japan'}

# プロットの設定
plt.figure(figsize=(12, 12))

# 原産国ごとにデータをプロット
for origin_value, marker in markers.items():
    # originがorigin_valueのインデックスを取得
    indices = np.where(origins == origin_value)
    # データを取得してプロット
    plt.scatter(X_reduced[indices, 0], X_reduced[indices, 1],
                marker=marker, label=f'{place[origin_value]}')

car_names = df['car name']

# X_reduced: shape = (n_samples, 2) のPCスコア行列（第1, 第2主成分）
x = X_reduced[:, 0]
y = X_reduced[:, 1]

# 各点に車名を注記（少しだけ右上へずらす）
"""
for xi, yi, label in zip(x, y, car_names):
    plt.annotate(label, (xi, yi),
                xytext=(2, 2), textcoords='offset points',
                fontsize=8, alpha=0.9)
"""

# 主成分負荷量（ロードイング）の取得
loading_scores = eigen_vectors[:, :k] * np.sqrt(eigen_values[:k])

# 特徴量名
feature_names = features

# ロードイングのスケーリング（視覚的にわかりやすくするため）
scaling_factor = 1  # 適宜調整可能

# ロードイングをプロット
for i in range(len(feature_names)):
    plt.arrow(0, 0, loading_scores[i, 0]*scaling_factor, loading_scores[i, 1]*scaling_factor,
              color='red', width=0.01, head_width=0.1, alpha=0.7, zorder=3)
    plt.text(loading_scores[i, 0]*scaling_factor*1.5, loading_scores[i, 1]*scaling_factor*1.5,
             feature_names[i], color='red', ha='center', va='center', fontsize=12, zorder=3)

# 軸ラベルとタイトル
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

# 凡例とグリッド
plt.legend()
plt.grid()

# プロットの表示
plt.show()

In [ ]:
# 寄与率の計算
explained_variance_ratio = eigen_values / np.sum(eigen_values)
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

# 寄与率を表示
for i, (evr, cv) in enumerate(zip(explained_variance_ratio, cumulative_variance_ratio)):
    print(f'Principal Component {i+1}: Explained Variance Ratio = {evr:.4f}, Cumulative = {cv:.4f}')

In [ ]:
# ロードイングをデータフレームにまとめる
loading_matrix = pd.DataFrame(loading_scores, columns=[f'PC{i+1}' for i in range(k)], index=feature_names)

# ロードイングを表示
print(loading_matrix)